In [99]:
%reset -f 

In [100]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate


from sklearn import preprocessing 
from sklearn import metrics
%matplotlib inline

In [101]:
#Leer Los datos
df_train = pd.read_csv("data/train.csv", index_col="PassengerId")
df_test = pd.read_csv("data/test.csv", index_col="PassengerId")

In [102]:
#Borrar Variables
df_train.drop(columns=["Name","Cabin","Ticket"], inplace=True);
df_test.drop(columns=["Name","Cabin","Ticket"], inplace=True);

In [103]:
#Convertir a tipos de variable adecuados 
df_train.convert_dtypes();
df_test.convert_dtypes();

In [104]:
# Rellenar o Borrar valores NAN

df_train.fillna(df_train.median(), inplace=True)
#df_train.dropna(inplace=True)
df_test.fillna(df_train.median(), inplace=True)


In [105]:
#Reducir dimensionalidad encontrado correlación de variables
df_train["Family_members"] = df_train["SibSp"] + df_train["Parch"]
df_test["Family_members"] = df_test["SibSp"] + df_test["Parch"]
df_train.drop(columns=["SibSp","Parch"], inplace=True);
df_test.drop(columns=["SibSp","Parch"], inplace=True);

In [106]:
#Categorizar variables  (Sex y Embarked)
df_train[["Sex"]] = df_train[["Sex"]].astype("category")
df_test[["Sex"]] = df_test[["Sex"]].astype("category")

df_train[["Embarked"]] = df_train[["Embarked"]].astype("category")
df_test[["Embarked"]] = df_test[["Embarked"]].astype("category")

# Obtener dummis de variables categoricas 
df_train[["IsWomen","IsMan"]] = pd.get_dummies(df_train[["Sex"]])
df_test[["IsWomen","IsMan"]] = pd.get_dummies(df_test[["Sex"]])

df_train[["IsC","IsQ","IsS"]] = pd.get_dummies(df_train[["Embarked"]])
df_test[["IsC","IsQ","IsS"]] = pd.get_dummies(df_test[["Embarked"]])

# Borrar Dummies Sobrantes y Variables Categoricas Originales 
df_train.drop(columns=["IsMan","Sex","IsQ","Embarked"], inplace=True);
df_test.drop(columns=["IsMan","Sex","IsQ","Embarked"], inplace=True);

In [107]:
#Agrupar datos numericos 

#( ] ( ] ( ] ( ] ( ]

#0.8061
bins = [-10, 0, 5, 12, 32, 60, 100]
names = [1, 2, 3, 4, 5, 6] 

#Age 
df_train["Age"]=pd.cut( df_train["Age"], bins =bins, labels = names)
df_test["Age"]=pd.cut( df_test["Age"], bins =bins, labels = names )

df_train["Age"]=df_train.Age.astype('category').cat.codes
df_test["Age"]=df_test.Age.astype('category').cat.codes


In [108]:
df_train.Age.value_counts()

#df_train[df_train.Age==1]

3    554
4    246
1     44
2     25
5     22
Name: Age, dtype: int64

In [66]:
#Graficas
sns.boxplot(data = df_train._get_numeric_data())
 
#pair map
sns.pairplot(df_train, hue="Survived", plot_kws={'alpha':0.3} )

# corr 
corr= df_train._get_numeric_data().corr()
sns.heatmap(corr, yticklabels = corr.columns, xticklabels = corr.columns,  annot=True)

In [111]:
# Machine Learning

# X = Features Y = Target 
X = df_train.drop(columns=['Survived'])
Y = df_train['Survived']


In [68]:
Classifiers_dict = {    #"RandomForest"          :RandomForestClassifier(),
                        "GradientBoosting"      :GradientBoostingClassifier(n_estimators=110),
                        # "KNeighborsClassifier"  :KNeighborsClassifier(),
                        # "DecisionTreeClassifier":DecisionTreeClassifier(),
} 

In [69]:
for  Name, Classifier  in Classifiers_dict.items():
    print(Name)
    print()
    CrossResults = cross_validate(Classifier, X, Y, cv=5, return_train_score=True)
    print (f"Train Mean = {np.mean(CrossResults['train_score'])}")
    print (f"Test Mean = {np.mean(CrossResults['test_score'])}" )
    print (f"Train MAX = {np.max(CrossResults['train_score'])}")
    print (f"Test MAX = {np.max(CrossResults['test_score'])}" )
    
    estimator = Classifier.fit(X,Y)
    Y_test=estimator.predict(df_test)


    try :
        print (list(zip(X, np.round(estimator.feature_importances_,decimals=4))))
    except:
        pass

    df_salida = df_test.copy(deep=True)
    df_salida['Survived'] = Y_test
    df_salida.to_csv(f"data/AAA__Titanic_{Name}_.csv", index=True, columns=["Survived"])

    print("---"*100)


GradientBoosting

Train Mean = 0.8964621712340641
Test Mean = 0.8305504990270542
Train MAX = 0.9144460028050491
Test MAX = 0.8820224719101124
[('Pclass', 0.1404), ('Age', 0.0726), ('Fare', 0.1998), ('Family_members', 0.0884), ('IsWomen', 0.4815), ('IsC', 0.0032), ('IsS', 0.0141)]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [70]:
GradientBoostingClassifier()

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [92]:
param_test1 = { "learning_rate":[0.1,0.05,0.025,0.01,0.005,0.001],
                "n_estimators": range(100,1000,100),
                "max_depth": range(3,5,1),
                }

estimator = GradientBoostingClassifier(min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)

gsearch1 = GridSearchCV( estimator,
                         param_grid = param_test1,
                         scoring="accuracy",
                         n_jobs=4,
                         iid=False,
                         cv=5)

gsearch1.fit(X, Y) 
         

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no...nge=None,
        

In [93]:
print ( f" Grid: {gsearch1.best_params_} \n Best Score{gsearch1.best_score_}" )
final_results = cross_validate(gsearch1.best_estimator_, X , Y, cv=5, return_train_score=True)
print("")
print (f"Train Mean = {np.mean(final_results['train_score'])}")
print (f"Test Mean = {np.mean(final_results['test_score'])}" )
print (f"Train MAX = {np.max(final_results['train_score'])}")
print (f"Test MAX = {np.max(final_results['test_score'])}" )



Grid: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 900} 
 Best Score0.8406377502981608

Train Mean = 0.9169461209953196
Test Mean = 0.8406377502981608
Train MAX = 0.9256661991584852
Test MAX = 0.8707865168539326


In [115]:
final_estimator = GradientBoostingClassifier(n_estimators=900,
                                             learning_rate=0.01,
                                             max_depth=3)
final_estimator.fit(X,Y)

Y_test=final_estimator.predict(df_test)

df_salida = df_test.copy(deep=True)
df_salida['Survived'] = Y_test
df_salida.to_csv("data/ABB_GradientBoostingClassifier.csv", index=True, columns=["Survived"])

In [98]:
df_test=df_test.drop["Survived"]

NameError: name 'df_test' is not defined